In [ ]:
import torch
import numpy as np
import torchvision as tv
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image
import cv2

from model.networks_tf import Generator, Discriminator

In [6]:
use_cuda_if_available = True
device = torch.device('cuda' if torch.cuda.is_available() 
                             and use_cuda_if_available else 'cpu')
generator = Generator(cnum_in=5, cnum=48, return_flow=True).to(device)

generator_state_dict = torch.load('pretrained/states_tf_celebahq.pth')['G']
generator.load_state_dict(generator_state_dict)

<All keys matched successfully>

In [8]:
for n in range(1, 50):
    try:
        n = str(n).zfill(6)
        image = r"D:\frank\paper\RAFT\demo-frames\lulabtop1\{}.png".format(n)
        mask  = r"D:\frank\paper\RAFT\demo-frames\final_mask\{}.png".format(n)

        image = Image.open(image)
        mask = Image.open(mask)

        # plt.figure(figsize=(15, 10))
        # plt.subplot(1, 2, 1)
        # plt.imshow(image)  # plot raw image
        # plt.subplot(1, 2, 2)
        # plt.imshow(mask)   # plot masked image

        # preprocessing
        image_org = T.ToTensor()(image)
        mask = T.ToTensor()(mask)

        _, h, w = image_org.shape
        grid = 8

        image = image_org[:3, :h//grid*grid, :w//grid*grid].unsqueeze(0)
        mask = mask[0:1, :h//grid*grid, :w//grid*grid].unsqueeze(0)

        print(f"Shape of image: {image.shape}")

        image = (image*2 - 1.).to(device)  # map image values to [-1, 1] range
        mask = (mask > 0.).to(dtype=torch.float32, device=device)  # 1.: masked 0.: unmasked

        image_masked = image * (1.-mask)  # mask image

        ones_x = torch.ones_like(image_masked)[:, 0:1, :, :]
        x = torch.cat([image_masked, ones_x, ones_x*mask], dim=1)  # concatenate channels


        with torch.no_grad():
            x_stage1, x_stage2, offset_flow = generator(x, mask)

        image_inpainted = image_masked * (1.-mask) + x_stage2 * mask


        def pt_to_rgb(pt): 
            return np.array(pt[0].cpu().permute(1, 2, 0)*0.5 + 0.5)

        print("Result:")
        # plt.figure(figsize=(10, 8))
        # plt.imshow(pt_to_rgb(image_inpainted))
        # print(pt_to_rgb(image_inpainted)*255)
        # plt.show()
        cv2.imwrite(r"D:\frank\paper\RAFT\demo-frames\final_result\{}.png".format(n), 
                    img=pt_to_rgb(image_inpainted)*255)
        
    except:
        print(n, " 沒有影像")

Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
000011  沒有影像
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3, 536, 960])
Result:
Shape of image: torch.Size([1, 3,